In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import math
import IPython, PIL
from IPython.display import Image

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# cap= cv2.VideoCapture("/content/drive/My Drive/Colab_Notebooks/datasets/sample_video.mp4")
# i = 0
# while (cap.isOpened()) :
#     ret, frame = cap.read()
#     if ret == False :
#         break
#     path = os.path.join(dirname, 'tvimg'+str(i)+'.jpg')
#     cv2.imwrite(path, frame)
#     i = i + 1
 
# cap.release()
# cv2.destroyAllWindows()

In [14]:
# where the frames from video will be saved
dirname = "/content/drive/My Drive/Colab_Notebooks/Outputs/virtual trainer/frames"

In [15]:
#where the images with skeleton will be saved
processed = "/content/drive/My Drive/Colab_Notebooks/Outputs/virtual trainer/frames+skel"

In [16]:
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

MODE = "COCO"

if MODE is "COCO":
    protoFile = "/content/drive/My Drive/Colab_Notebooks/models/pose/coco/pose_deploy_linevec.prototxt"
    weightsFile = "/content/drive/My Drive/Colab_Notebooks/models/pose_iter_440000.caffemodel"
    nPoints = 18
    POSE_PAIRS = [ [1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]

# MODE = "MPI"

# if MODE is "MPI" :
#     protoFile = "/content/drive/My Drive/Colab_Notebooks/models/pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt"
#     weightsFile = "/content/drive/My Drive/Colab_Notebooks/models/pose_iter_160000.caffemodel"
#     # nPoints = 20
#     # POSE_PAIRS = [ [12, 5], [4, 5], ] # for hands
#     nPoints = 15
#     POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

In [17]:
def draw_skeleton_on_image(_frame) :

  frame = _frame
  frameCopy = np.copy(frame)
  frameWidth = frame.shape[1]
  frameHeight = frame.shape[0]
  threshold = 0.1

  inWidth = 368
  inHeight = 368

  inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                          (0, 0, 0), swapRB=False, crop=False)

  net.setInput(inpBlob)

  output = net.forward()
  H = output.shape[2]
  W = output.shape[3]

  # Empty list to store the detected keypoints
  points = []; points2 = []

  for i in range(nPoints):
    # confidence map of corresponding body's part.
    probMap = output[0, i, :, :]

    # Find global maxima of the probMap.
    minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
    
    # Scale the point to fit on the original image
    x = (frameWidth * point[0]) / W
    y = (frameHeight * point[1]) / H

    if prob > threshold:
        if (i==5 or i==6 or i==7): 
            cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
            cv2.circle(frame, (int(x), int(y)), 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            # cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
            # cv2.putText(frame, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
            points2.append((int(x), int(y)))
        else:
            points2.append(None)

        # Add the point to the list if the probability is greater than the threshold
        points.append((int(x), int(y)))
    else :
        points.append(None)
        points2.append(None)

  # Draw Skeleton
  for pair in POSE_PAIRS:
      partA = pair[0]
      partB = pair[1]

      if points2[partA] and points2[partB]:
          cv2.line(frame, points2[partA], points2[partB], (0, 255, 255), 3)

  plt.figure(figsize=[10,10])
  #plt.imshow(cv2.cvtColor(frameCopy, cv2.COLOR_BGR2RGB))
  plt.figure(figsize=[10,10])
  #plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
  dirr = os.path.join(processed, "skeleton{}.jpg".format(kount))
  cv2.imwrite(dirr, frame)

In [19]:
import glob
# take all the frames
images = glob.glob("/content/drive/My Drive/Colab_Notebooks/Outputs/virtual trainer/frames" + '/*.jpg')

In [20]:
images.sort()
p = len(images)
p

595

In [22]:
kount = 88 #I first took images from 88th frame
for cnt in range(88, p):
  image_name = "tvimg{}.jpg".format(cnt)
  image_path = os.path.join("/content/drive/My Drive/Colab_Notebooks/Outputs/virtual trainer/frames", image_name)
  frame = cv2.imread(image_path)
  draw_skeleton_on_image(frame)
  kount += 1
  #print(image_path)
  if cnt%50==0:
      print("upto",image_name,"images done")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


upto tvimg100.jpg images done
upto tvimg150.jpg images done
upto tvimg200.jpg images done
upto tvimg250.jpg images done
upto tvimg300.jpg images done
upto tvimg350.jpg images done
upto tvimg400.jpg images done
upto tvimg450.jpg images done
upto tvimg500.jpg images done
upto tvimg550.jpg images done


<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

...

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

In [23]:
img_array = []
for cnt in range(90, p) :
    image_name = "skeleton{}.jpg".format(cnt)
    filename = os.path.join(processed, image_name)
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img) 
    # print(image_name)
    #print(filename)
 
out = cv2.VideoWriter('weightlift_skel_coco.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)) :
    out.write(img_array[i])
out.release()

<h2> Remove the human </h3>

In [24]:
import cv2
import numpy as np
import math
import IPython, PIL
from IPython.display import Image

### Only Skeleton Function

In [25]:
def only_skeleton(image):
    lower_range = np.array([0, 0,255])  # Set the Lower range value of color in BGR
    upper_range = np.array([30,255,255])   # Set the Upper range value of color in BGR
    mask = cv2.inRange(image,lower_range,upper_range) # Create a mask with range
    thresh = cv2.bitwise_and(image,image,mask = mask)  # Performing bitwise and operation with mask in img variable

    image = thresh
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel_size = 5
    blur_gray = cv2.GaussianBlur(gray,(kernel_size,kernel_size),0)

    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    # create a mask of the edges image using cv2.filpoly()
    mask = np.zeros_like(edges)
    ignore_mask_color = 255

    # define the Region of Interest (ROI) - source code sets as a trapezoid for roads
    imshape = image.shape

    vertices = np.array([[(imshape[0], 0),(0, 0), (0, imshape[1]),(imshape[0],imshape[1])]], dtype=np.int32)

    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)

    return masked_edges

In [26]:
# import glob
# skeletons = glob.glob("/content/drive/My Drive/Colab_Notebooks/Outputs/virtual trainer/frames+skel" + '/*.jpg')

In [27]:
# skeletons.sort()

In [28]:
# # i will start taking frames from 88
# for cnt in range(88, p):
#   image_name = "skeleton{}.jpg".format(cnt)
#   image_path = os.path.join(processed, image_name)
#   frame = cv2.imread(image_path)
#   frame = only_skeleton(frame)
#   dirr = os.path.join("/content/drive/My Drive/Colab_Notebooks/Outputs/vitrtual trainer/only_skel_coco", "only_skel{}.jpg".format(kount))
#   cv2.imwrite(dirr, frame)

#   #print(image_name)
#   if cnt%50==0:
#     print("upto",image_name,"images done")

In [29]:
# len(os.listdir("/content/drive/My Drive/Colab_Notebooks/Outputs/virtual trainer/only_skel"))

In [30]:
# img_array = []
# kount =  len(os.listdir("/content/drive/My Drive/Colab_Notebooks/Outputs/openpose only skel"))-1
# kk = 93
# for cnt in range(kount) :
#     image_name = "only_skel{}.jpg".format(kk)
#     filename = os.path.join("/content/drive/My Drive/Colab_Notebooks/Outputs/openpose only skel", image_name)
#     img = cv2.imread(filename)
#     height, width, layers = img.shape
#     size = (width,height)
#     img_array.append(img) 
#     #print(image_name)
#     #print(filename)
#     kk = kk + 1
 
# out = cv2.VideoWriter('Only Skeleton.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
# for i in range(len(img_array)) :
#     out.write(img_array[i])
# out.release()

<h3>Angles </h3>

In [31]:
from PIL import Image
import numpy as np

def angles_count(image):
    masked_edges = only_skeleton(image)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel_size = 5
    blur_gray = cv2.GaussianBlur(gray,(kernel_size,kernel_size),0)

    lower_range = np.array([0, 0,255])  # Set the Lower range value of color in BGR
    upper_range = np.array([30,255,255])   # Set the Upper range value of color in BGR
    mask = cv2.inRange(image,lower_range,upper_range) # Create a mask with range
    thresh = cv2.bitwise_and(image,image,mask = mask)  # Performing bitwise and operation with mask in img variable

    # define the Hough Transform parameters
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 60 #minimum number of pixels making up a line
    max_line_gap = 5    # maximum gap in pixels between connectable line segments

    # make a blank the same size as the original image to draw on
    line_image = np.copy(image)*0 
    # IPython.display.display(PIL.Image.fromarray(line_image))

    # run Hough on edge detected image
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)

    abs_angle1, abs_angle2, abs_angle = 0, 0, 0
    if lines is None:
        return None, abs_angle
    temp1, temp2 = 0, 360
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(0,0,255),10)
            angle = math.atan2(x2-x1, y2-y1)
            angle = angle * 180 / 3.14
            
            abs_angle1 = max(temp1, abs(180 - angle))
            abs_angle2 = min(temp2, abs(180 - angle))
            temp1 = abs_angle1
            temp2 = abs_angle2

            # print(temp1, temp2)

    # draw the line on the original image

    thresh2 = Image.fromarray(thresh, 'RGB')
    # img = Image.fromarray(image, mode='L')
    lines_edges = cv2.addWeighted(image, 0.8, line_image, 1, 0)

    position_angle = ((int) (thresh.shape[1]/2 - 268/2 + 290), (int) (lines_edges.shape[0]/2 - 36/2 - 30))
    position_count = ((int) (thresh.shape[1]/2 - 268/2 + 290), (int) (lines_edges.shape[0]/2 - 36/2 + 60))

    if 0<=abs_angle1<=90:
        abs_angle = abs_angle1
    elif 0<=abs_angle2<=90:
        abs_angle = abs_angle2
    # print("Angle = ", abs_angle)
    cv2.putText(lines_edges, "Angle: {}".format(abs_angle), position_angle, cv2.FONT_HERSHEY_SIMPLEX, 1, (139, 0, 0), 2)

    return lines_edges, abs_angle

    # cv2.imwrite("lanes_detected.jpg", lines_edges)
    # cv2.destroyAllWindows()

In [37]:
# i will start taking frames from 88
start = 0; count = 0
for cnt in range(90, 595):
    image_name = "skeleton{}.jpg".format(cnt)
    image_path = os.path.join(processed, image_name)
    frame = cv2.imread(image_path)
    a, abs_angle = angles_count(frame)

    position_angle = ((int) (frame.shape[1]/2 - 268/2 + 290), (int) (frame.shape[0]/2 - 36/2 - 30))
    position_count = ((int) (frame.shape[1]/2 - 268/2 + 290), (int) (frame.shape[0]/2 - 36/2 + 60))

    dirr = os.path.join("/content/drive/My Drive/Colab_Notebooks/Outputs/virtual trainer/final", "final{}.jpg".format(cnt))

    if a is None:
        cv2.putText(frame, "Angle: 0", position_angle, cv2.FONT_HERSHEY_SIMPLEX, 1, (139, 0, 0), 2)
        cv2.putText(frame, "{}".format(count), position_count, cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2)
        cv2.imwrite(dirr, frame)
        continue
    else:
        #counting
        if 0<abs_angle<7:
            start = 1
        if start==1 and float(17)<abs_angle<float(18):
            count += 1
            start = 0
    cv2.putText(a, "# {}".format(count), position_count, cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2)  
    cv2.imwrite(dirr, a)
    # print(abs_angle, start, count)
        
    #print(image_name)
    if cnt%50==0:
        print("upto",image_name,"images done")

upto skeleton100.jpg images done
upto skeleton150.jpg images done
upto skeleton200.jpg images done
upto skeleton250.jpg images done
upto skeleton300.jpg images done
upto skeleton350.jpg images done
upto skeleton400.jpg images done
upto skeleton450.jpg images done
upto skeleton500.jpg images done
upto skeleton550.jpg images done


In [42]:
img_array = []
final = "/content/drive/My Drive/Colab_Notebooks/Outputs/virtual trainer/final"
kk = 93 # i will start taking frames from 93
for cnt in range(90, p) :
    image_name = "final{}.jpg".format(cnt)
    filename = os.path.join(final, image_name)
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img) 
    # print(image_name)
    #print(filename)
 
out = cv2.VideoWriter('weightlift_final_COCO2.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
 
for i in range(len(img_array)) :
    out.write(img_array[i])
out.release()
print("Done")

Done
